## numpy

In [1]:
import numpy as np

In [2]:
N, D_in, H, D_out = 64, 1000, 100, 10

In [7]:
# Nはデータ数
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

# x = torch.randn(N, D_in)
# y = torch.randn(N, D_out)

In [4]:
w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

In [12]:
lr = 1e-6

for t in range(500):
    # (64, 1000) * (1000, 100) = (64, 100)
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    # (64, 100) * (100, 10) = (64, 10)
    y_pred = h_relu.dot(w2)
    # 残差平方和
    loss = np.square(y_pred - y).sum()
    if t % 100 == 99:
        print(t, loss)
    
    # 微分値
    grad_y_pred = 2.0 * (y_pred - y)
    # (100, 64) * (64, 10) = (100, 10)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    # (64, 10) * (10, 100) = (64, 100)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    # reluで0になったところは、勾配も0
    grad_h[h < 0] = 0
    #  (1000, 64) * (64, 100) = (1000, 100)
    grad_w1 = x.T.dot(grad_h)
    
    w1 -= grad_w1 * lr
    w2 -= grad_w2 * lr

99 1.7560606753075396e-06
199 4.229921551387729e-08
299 1.0555740284998412e-09
399 2.703305323051672e-11
499 7.054137054685965e-13


## Tensors

In [13]:
import torch

In [14]:
dtype = torch.float
device = torch.device('cpu')

In [15]:
N, D_in, H, D_out = 64, 1000, 100, 10

In [16]:
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

In [25]:
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

In [34]:
lr = 1e-6

for t in range(500):
    # mm ? 
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)
    
    # tensor.item() -> float
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)
    
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    # Update weights using gradient descent
    w1 -= lr * grad_w1
    w2 -= lr * grad_w2

99 439.7330627441406
199 2.0890729427337646
299 0.015256239101290703
399 0.00031060833134688437
499 4.674466617871076e-05


## NN

In [33]:
import torch
import random

N is batch size  
D_in is input feature dimentions

In [2]:
N, D_in, H, D_out = 64, 1000, 100, 10

In [3]:
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

In [8]:
x.size(), y.size()

(torch.Size([64, 1000]), torch.Size([64, 10]))

In [4]:
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

In [5]:
loss_fn = torch.nn.MSELoss(reduction='sum')

In [15]:
lr = 1e-4

# 同じbatchを500回、学習させている
for t in range(500):
    # １つ目のサンプルは、〜、みたいのがたくさん入っている
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    
    if t % 100 == 99:
        print(t, loss.item())
    
    model.zero_grad()
    
    loss.backward()
    
    # weight更新
    with torch.no_grad():
        for param in model.parameters():
            param -= lr * param.grad

99 4.942885545311526e-10
199 3.4711028495948426e-10
299 2.6340421510617773e-10
399 2.0833654246210642e-10
499 1.6762380372625785e-10


## optim

In [2]:
N, D_in, H, D_out = 64, 1000, 100, 10

In [3]:
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

In [17]:
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
loss_fn = torch.nn.MSELoss(reduction='sum')

In [19]:
lr = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
for t in range(500):
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

99 55.200584411621094
199 0.8628503084182739
299 0.004844253417104483
399 5.791613875771873e-05
499 4.2589658733049873e-07


## Custom nn Modules

In [22]:
class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)
        
    def forward(self, x):
        # clamp(min=0) == relu
        return self.linear2(self.linear1(x).clamp(min=0))

In [23]:
N, D_in, H, D_out = 64, 1000, 100, 10

In [24]:
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

In [25]:
model = TwoLayerNet(D_in, H, D_out)

In [26]:
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

for t in range(500):
    y_pred = model(x)
    
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

99 1.7557014226913452
199 0.019042925909161568
299 0.00037909974344074726
399 9.968287486117333e-06
499 3.1555356372336973e-07


バッチを上限いっぱいにすれば、ちゃんと学習できそう

## Control Flow + Weight Sharing

In [28]:
class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D_out)
        
    def forward(self, x):
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0, 3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        return y_pred

In [29]:
N, D_in, H, D_out = 64, 1000, 100, 10

In [30]:
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

In [35]:
x.size(), y.size()

(torch.Size([64, 1000]), torch.Size([64, 10]))

In [31]:
model = DynamicNet(D_in, H, D_out)

In [34]:
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)

for t in range(500):
    y_pred = model(x)
    
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

99 81.3045425415039
199 0.7954521775245667
299 2.7631566524505615
399 2.1098971366882324
499 0.8980854153633118
